BLOCK 1: Custom Decision Tree Implementation (Iris Dataset)

In [6]:
# Custom Decision Tree Implementation (Iris Dataset)

import numpy as np

class CustomDecisionTree:
    def __init__(self, max_depth=None):
        """
        Initialize the decision tree.
        Parameters:
        max_depth: int or None
            Maximum depth of the tree.
        """
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        """
        Train the decision tree on training data.
        """
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        """
        Recursively build the tree based on information gain.
        """
        num_samples, num_features = X.shape
        unique_classes = np.unique(y)

        # Stopping condition 1: all samples are of same class
        if len(unique_classes) == 1:
            return {'class': unique_classes[0]}

        # Stopping condition 2: reached maximum depth
        if num_samples == 0 or (self.max_depth and depth >= self.max_depth):
            return {'class': np.bincount(y).argmax()}

        best_info_gain = -float('inf')
        best_split = None

        # Iterate over all features and thresholds
        for feature_idx in range(num_features):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                left_mask = X[:, feature_idx] <= threshold
                right_mask = ~left_mask
                left_y = y[left_mask]
                right_y = y[right_mask]

                info_gain = self._information_gain(y, left_y, right_y)
                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_split = {
                        'feature_idx': feature_idx,
                        'threshold': threshold,
                        'left_mask': left_mask,
                        'right_mask': right_mask
                    }

        if best_split is None:
            return {'class': np.bincount(y).argmax()}

        # Recursively build left and right subtrees
        left_tree = self._build_tree(X[best_split['left_mask']], y[best_split['left_mask']], depth + 1)
        right_tree = self._build_tree(X[best_split['right_mask']], y[best_split['right_mask']], depth + 1)

        return {
            'feature_idx': best_split['feature_idx'],
            'threshold': best_split['threshold'],
            'left_tree': left_tree,
            'right_tree': right_tree
        }

    def _information_gain(self, parent, left, right):
        """
        Calculate information gain of a split.
        """
        parent_entropy = self._entropy(parent)
        left_entropy = self._entropy(left)
        right_entropy = self._entropy(right)

        weighted_avg = (len(left)/len(parent)) * left_entropy + (len(right)/len(parent)) * right_entropy
        return parent_entropy - weighted_avg

    def _entropy(self, y):
        """
        Calculate entropy of a label set.
        """
        if len(y) == 0:
            return 0
        probs = np.bincount(y) / len(y)
        return -np.sum([p * np.log2(p + 1e-9) for p in probs if p > 0])

    def predict(self, X):
        """
        Predict class labels for a dataset.
        """
        return np.array([self._predict_single(x, self.tree) for x in X])

    def _predict_single(self, x, tree):
        """
        Predict class label for a single sample.
        """
        if 'class' in tree:
            return tree['class']

        feature_val = x[tree['feature_idx']]
        if feature_val <= tree['threshold']:
            return self._predict_single(x, tree['left_tree'])
        else:
            return self._predict_single(x, tree['right_tree'])


BLOCK 2: Load Iris Dataset and Split

In [7]:
# Load Iris Dataset and Split into Train/Test

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split dataset: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training samples:", X_train.shape[0])
print("Test samples:", X_test.shape[0])


Training samples: 120
Test samples: 30


BLOCK 3: Train & Evaluate Custom Decision Tree

In [8]:
# Train and Evaluate Custom Decision Tree

from sklearn.metrics import accuracy_score

# Initialize custom decision tree with max_depth=3
custom_tree = CustomDecisionTree(max_depth=3)
custom_tree.fit(X_train, y_train)

# Predict on test data
y_pred_custom = custom_tree.predict(X_test)

# Calculate accuracy
accuracy_custom = accuracy_score(y_test, y_pred_custom)
print(f"Custom Decision Tree Accuracy: {accuracy_custom:.4f}")


Custom Decision Tree Accuracy: 1.0000


BLOCK 4: Train & Evaluate Scikit-learn Decision Tree

In [9]:
# Train and Evaluate Scikit-learn Decision Tree

from sklearn.tree import DecisionTreeClassifier

# Initialize scikit-learn decision tree
sklearn_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
sklearn_tree.fit(X_train, y_train)

# Predict on test data
y_pred_sklearn = sklearn_tree.predict(X_test)

# Calculate accuracy
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
print(f"Scikit-learn Decision Tree Accuracy: {accuracy_sklearn:.4f}")

# Compare results
print("\nAccuracy Comparison:")
print(f"Custom Decision Tree: {accuracy_custom:.4f}")
print(f"Scikit-learn Decision Tree: {accuracy_sklearn:.4f}")


Scikit-learn Decision Tree Accuracy: 1.0000

Accuracy Comparison:
Custom Decision Tree: 1.0000
Scikit-learn Decision Tree: 1.0000


BLOCK 5: Ensemble Methods (Random Forest) – Wine Dataset

In [24]:
# Classification Models: Wine Dataset
# Decision Tree & Random Forest

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

wine = load_wine()
X_w, y_w = wine.data, wine.target

X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(X_w, y_w, test_size=0.2, random_state=42)

# Decision Tree
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train_w, y_train_w)
y_pred_dt = dt_classifier.predict(X_test_w)
f1_dt = f1_score(y_test_w, y_pred_dt, average='weighted')
print(f"Decision Tree F1 Score: {f1_dt:.4f}")

# Random Forest
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train_w, y_train_w)
y_pred_rf = rf_classifier.predict(X_test_w)
f1_rf = f1_score(y_test_w, y_pred_rf, average='weighted')
print(f"Random Forest F1 Score: {f1_rf:.4f}")




Decision Tree F1 Score: 0.9440
Random Forest F1 Score: 1.0000


BLOCK 6: Hyperparameter Tuning – Random Forest Classifier (GridSearchCV)

In [25]:
# Hyperparameter Tuning: Random Forest Classifier

from sklearn.model_selection import GridSearchCV

# Define hyperparameter grid (correct for scikit-learn >=1.1)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='f1_weighted',
    n_jobs=-1
)

# Fit grid search
grid_search.fit(X_train_w, y_train_w)

# Best parameters and evaluation
print("Best Parameters (Random Forest Classifier):", grid_search.best_params_)
best_rf = grid_search.best_estimator_
y_pred_best_rf = best_rf.predict(X_test_w)
f1_best_rf = f1_score(y_test_w, y_pred_best_rf, average='weighted')
print(f"Best Random Forest F1 Score: {f1_best_rf:.4f}")


Best Parameters (Random Forest Classifier): {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
Best Random Forest F1 Score: 1.0000


BLOCK 7: Regression with Decision Tree and Random Forest

In [30]:
# Decision Tree Regressor and Random Forest Regressor

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd

# Load California Housing dataset from OpenML
housing = fetch_openml(name="california_housing", version=1, as_frame=True)

# Features and target
X = housing.data.copy()          # DataFrame of features
y = housing.target.copy().astype(float)  # Target as float

# Combine into one DataFrame for feature engineering
df = X.copy()
df["MedHouseVal"] = y  # Explicitly name target column

# Feature Engineering: Create average-based features
df["AveRooms"]  = df["total_rooms"] / df["households"]
df["AveBedrms"] = df["total_bedrooms"] / df["households"]
df["AveOccup"]  = df["population"] / df["households"]

# Select relevant features for modeling
df = df[
    [
        "median_income",
        "housing_median_age",
        "AveRooms",
        "AveBedrms",
        "population",
        "AveOccup",
        "latitude",
        "longitude",
        "MedHouseVal"
    ]
]

# Remove missing values if any
df.dropna(inplace=True)

# Split features and target
X_r = df.drop("MedHouseVal", axis=1).astype(float)
y_r = df["MedHouseVal"].astype(float)

# Train-test split (80% train, 20% test)
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    X_r, y_r, test_size=0.2, random_state=42
)

# Train Decision Tree Regressor
dt_regressor = DecisionTreeRegressor(random_state=42)
dt_regressor.fit(X_train_r, y_train_r)
y_pred_dt = dt_regressor.predict(X_test_r)
mse_dt = mean_squared_error(y_test_r, y_pred_dt)

# Train Random Forest Regressor
rf_regressor = RandomForestRegressor(random_state=42)
rf_regressor.fit(X_train_r, y_train_r)
y_pred_rf = rf_regressor.predict(X_test_r)
mse_rf = mean_squared_error(y_test_r, y_pred_rf)

print("Regression Models Evaluation:")
print(f"Decision Tree Regressor MSE: {mse_dt:.4f}")
print(f"Random Forest Regressor MSE: {mse_rf:.4f}")


Regression Models Evaluation:
Decision Tree Regressor MSE: 5499044142.5924
Random Forest Regressor MSE: 2683756555.1180


BLOCK 8: Hyperparameter Tuning (Random Forest Regressor)

In [31]:
# Using RandomizedSearchCV

from sklearn.model_selection import RandomizedSearchCV

# Define hyperparameters to tune
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# RandomizedSearchCV for Random Forest Regressor
random_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=10,            # number of random combinations to try
    cv=5,                 # 5-fold cross-validation
    scoring='neg_mean_squared_error',  # minimize MSE
    random_state=42,
    n_jobs=-1             # use all processors
)

# Fit RandomizedSearchCV
random_search.fit(X_train_r, y_train_r)

# Best hyperparameters
best_params_rf = random_search.best_params_
best_rf_reg = random_search.best_estimator_

# Predict using the best Random Forest Regressor
y_pred_best_rf = best_rf_reg.predict(X_test_r)
mse_best_rf = mean_squared_error(y_test_r, y_pred_best_rf)

print("Random Forest Regressor Hyperparameter Tuning Results:")
print(f"Best Hyperparameters: {best_params_rf}")
print(f"Best Random Forest Regressor MSE on Test Set: {mse_best_rf:.4f}")


Random Forest Regressor Hyperparameter Tuning Results:
Best Hyperparameters: {'n_estimators': 100, 'min_samples_split': 5, 'max_depth': 30}
Best Random Forest Regressor MSE on Test Set: 2686814150.2137


Key Points :

* Decision Trees split data using feature-based thresholds and can be implemented both from scratch and using Scikit-learn.

* Scikit-learn’s Decision Tree performs better due to optimized algorithms and pruning.

* Random Forest is an ensemble method that combines multiple decision trees to improve accuracy and reduce overfitting.

* F1 Score is an effective metric for multi-class classification as it balances precision and recall.

* GridSearchCV performs exhaustive hyperparameter tuning using cross-validation for classification models.

* RandomizedSearchCV is computationally efficient for tuning regression models with large datasets.

* In regression, Random Forest Regressor achieves lower MSE than Decision Tree Regressor due to ensemble averaging.

* Large MSE values occur because the target variable is large and errors are squared.

* Feature scaling is not required for tree-based models since they use threshold comparisons, not distance metrics.

* Hyperparameter tuning improves model generalization and performance stability.